In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.part'):
    shutil.rmtree('./file/3.part')

os.makedirs('./file/3.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(ua=False),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200 or resp.status_code == 404:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                if resp.status_code == 404:
                    raise

                # = = = = = = = = = = = = = = =

                dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

                # = = = = = = = = = = = = = = =

                if input_.loc[a, 'Url'].startswith('https://www.ebay.com/'):
                    market, country, language, locale = 'EBAY-US', 'US', 'en', 'en-US'
                elif input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
                    market, country, language, locale = 'EBAY-DE', 'DE', 'de', 'de-DE'
                elif input_.loc[a, 'Url'].startswith('https://www.ebay.co.uk/'):
                    market, country, language, locale = 'EBAY-GB', 'GB', 'en', 'en-GB'
                elif input_.loc[a, 'Url'].startswith('https://www.ebay.com.au/'):
                    market, country, language, locale = 'EBAY-AU', 'AU', 'en', 'en-AU'
                elif input_.loc[a, 'Url'].startswith('https://www.ebay.ca/'):
                    market, country, language, locale = 'EBAY-ENCA', 'CA', 'en', 'en-CA'
                
                dict_['o']['g']['marketplaceId'].strip() != market or dict_['o']['g']['country'].strip() != country or dict_['o']['g']['language'].strip() != language or dict_['o']['g']['locale'].strip() != locale or dict_['o']['g']['itemId'].strip() != input_.loc[a, 'Item Number']
                                
                # = = = = = = = = = = = = = = =

                index_model = -1
                for i in range(len(dict_['o']['w'])):
                    if 'model' in dict_['o']['w'][i][2]:
                        index_model = i
                        break
                
                # = = = = = = = = = = = = = = =

                if index_model == -1:
                    raise
                
                # = = = = = = = = = = = = = = =

                dict_compatibility = dict_['o']['w'][index_model][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'][-1] if 'COMPATIBILITY_TABLE' in dict_['o']['w'][index_model][2]['model']['modules'] else {}
                dict_compatibility = {'api': dict_compatibility['action']['URL'],
                                      'page': int(dict_compatibility['text'].strip()),
                                      'data': {'scopedContext': {'catalogDetails': {'categoryId': dict_compatibility['action']['params']['categoryId'],
                                                                                    'itemId': dict_compatibility['action']['params']['itemId'],
                                                                                    'marketplaceId': dict_compatibility['action']['params']['marketplaceId'],
                                                                                    'sellerName': dict_compatibility['action']['params']['sellerName']}}}} if dict_compatibility else {}

                # = = = = = = = = = = = = = = =

                dict_item, dict_count = dict_['o']['w'][index_model][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems'] if 'ABOUT_THIS_ITEM' in dict_['o']['w'][index_model][2]['model']['modules'] else {}, {}
                for value in dict_item.values():
                    key = value['labels'][0]['textSpans'][0]['text'].strip()
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1
                
                dict_specific, dict_no = {}, {}
                for i, value in enumerate(dict_item.values()):
                    key = value['labels'][0]['textSpans'][0]['text'].strip()
                    if dict_count[key] == 1:
                        dict_specific[str(i)] = {key: value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_specific[str(i)] = {f'''{key}-{dict_no[key]}''': value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

                # = = = = = = = = = = = = = = =

                description_url = dict_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip() if 'dataItems' in dict_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0] else dict_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

                # = = = = = = = = = = = = = = =

                title = dict_['o']['w'][index_model][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =

                subtitle = dict_['o']['w'][index_model][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in dict_['o']['w'][index_model][2]['model']['modules']['TITLE'] else ''

                # = = = = = = = = = = = = = = =

                price = dict_['o']['w'][index_model][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip() if 'binModel' in dict_['o']['w'][index_model][2]['model']['modules']['BUY_BOX'] else dict_['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =

                dict_quantity, list_quantity = {}, dict_['o']['w'][index_model][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in dict_['o']['w'][index_model][2]['model']['modules'] else []
                for i, quantity in enumerate(list_quantity):
                    dict_quantity[str(i)] = quantity['text'].strip()
            
                # = = = = = = = = = = = = = = =

                dict_src = {}
                list_media = dict_['o']['w'][index_model][2]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in dict_['o']['w'][index_model][2]['model']['modules']['PICTURE'] else []
                for i, media in enumerate(list_media):
                    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()
    
                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Item Number': input_.loc[a, 'Item Number'],
                                         'Market': country,
                                         'Title': title,
                                         'Price': price,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Description Url': description_url,
                                         'Subtitle': subtitle,
                                         'Json_Quantity': json.dumps(dict_quantity),
                                         'Json_Compatibility': json.dumps(dict_compatibility),
                                         'Json_Specific': json.dumps(dict_specific)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-item_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.part/{file}',
              f'''./file/3.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://www.ebay.com/itm/255922672592?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：5] - [当前时间：14:03:53]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://www.ebay.com/itm/253139036562?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：4] - [当前时间：14:03:54]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://www.ebay.com/itm/235653299500?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：3] - [当前时间：14:03:54]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://www.ebay.com/itm/255586148611?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：2] - [当前时间：14:03:54]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://www.ebay.com/itm/233621258535?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：1] - [当前时间：14:03:54]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 9.https://www.ebay.com/itm/256102649999?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：0] - [当前时间：14:03:56]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://www.ebay.com/itm/256421824484?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：0] - [当前时间：14:03:57]

[状态：o

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3733.25it/s]

Done ~
